# Load gdrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
root_dir="/content/drive/MyDrive/Semester-Docs/Sem5/Capstone/"
proj_dir=root_dir+"rsna-pneumonia-detection-challenge/"
rsna_train_dir=proj_dir+"stage_2_train_images/"
rsna_metadata_path=proj_dir+"stage_2_train_labels.csv"

In [3]:
import pickle
import os
import pandas as pd
# dataset = 'rsa_pneumonia.pkl_final'
dataset1 = 'chest_xray_pneumonia.pkl_final'
dataset2 = 'chest_xray_augment.pkl_final'
pkl_file_name = 'chest_xray_pneumonia_augment.pkl_final'

with open(proj_dir + dataset1, 'rb') as file:
  merged_df = pickle.load(file)
with open(proj_dir + dataset2, 'rb') as file:
  augment_df = pickle.load(file)
merged_df = pd.concat([merged_df, augment_df], ignore_index=True)
augment_df.to_pickle(proj_dir+pkl_file_name)

# Custom Augmentor

In [ ]:
!pip install Augmentor

In [ ]:
import os
import pandas as pd
import Augmentor

source_data_dir = '/content/drive/MyDrive/Semester-Docs/Sem5/Capstone/chest_xray/train/PNEUMONIA/'
output_data_dir = '/content/drive/MyDrive/Semester-Docs/Sem5/Capstone/chest_xray/augment/PNEUMONIA'
# output_data_dir = '/content/drive/MyDrive/Semester-Docs/Sem5/Capstone/rsna-pneumonia-detection-challenge/augment/PNEUMONIA'
os.makedirs(output_data_dir, exist_ok=True)
pipeline = Augmentor.Pipeline(source_directory=source_data_dir, output_directory=output_data_dir)

pipeline.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
pipeline.flip_left_right(probability=0.5)
pipeline.flip_top_bottom(probability=0.5)
pipeline.zoom_random(probability=0.5, percentage_area=0.8)
pipeline.flip_random(probability=0.5)
pipeline.rotate_random_90(probability=0.5)
pipeline.random_contrast(probability=0.5, min_factor=0.5, max_factor=1.5)

num_augmented_images = (int)(len(to_be_augment_pos_image_paths) * 1.5)

pipeline.sample(num_augmented_images)
print("Data augmentation completed.")

Initialised with 3875 image(s) found.
Output directory set to /content/drive/MyDrive/Semester-Docs/Sem5/Capstone/chest_xray/augment/PNEUMONIA.

Processing <PIL.Image.Image image mode=L size=1160x800 at 0x79B43950FBB0>: 100%|██████████| 5812/5812 [07:51<00:00, 12.34 Samples/s]

Data augmentation completed.


# Create DF from augment data

In [ ]:
import os
import numpy as np
import pandas as pd
import time
from pathlib import Path

output_data_dir = '/content/drive/MyDrive/Semester-Docs/Sem5/Capstone/chest_xray/augment'
def create_file_map_all(root_dir, limit=100):
    file_map_all = {0:{},1:{}} # Initialize an empty map
    contents = os.listdir(root_dir)
    directories = [entry for entry in contents if os.path.isdir(os.path.join(root_dir, entry))]
    if (len(directories) != 2):
      print("Please check the directories must be 2 and it should be ['NORMAL', 'PNEUMONIA']")
      return;

    for directory in directories:
      if directory == 'NORMAL':
        label=0
      elif directory == 'PNEUMONIA':
        label=1
      else:
        print("Directory should be ['NORMAL', 'PNEUMONIA']")
        return;
      new_root_dir = Path(root_dir + "/" + directory)
      files = [file for file in new_root_dir.iterdir() if file.is_file()]
      for filename_full_path in files:
        filename = Path(filename_full_path).name
        full_path = os.path.join(new_root_dir, filename)
        if len(file_map_all[label]) != limit:
          file_map_all[label][filename] = full_path
        else:
          break
    return file_map_all

start_time = time.time()
file_map_all = create_file_map_all(output_data_dir, 10)

augment_file_map_0 = file_map_all[0]
augment_file_map_1 = file_map_all[1]

augment_df_0 = pd.DataFrame(augment_file_map_0.items(), columns=['file_name', 'file_path'])
augment_label_0 = pd.DataFrame({'label': np.zeros(len(augment_df_0))})
augment_df_new_0 = pd.concat([augment_df_0, augment_label_0], axis=1)

augment_df_1 = pd.DataFrame(augment_file_map_1.items(), columns=['file_name', 'file_path'])
augment_label_1 = pd.DataFrame({'label': np.ones(len(augment_df_1))})
augment_df_new_1 = pd.concat([augment_df_1, augment_label_1], axis=1)

augment_df = pd.concat([augment_df_new_0, augment_df_new_1], ignore_index=True)

elapsed_time = time.time() - start_time
print(f"Time taken: {elapsed_time:.4f} seconds")

print(augment_df)

# Loading file and creating a dataframe

In [ ]:
import os
import numpy as np
import pandas as pd
import time
from pathlib import Path

# Get a list of all files in the directory
def create_file_map_all(root_dir, metadata, limit=100):
    file_map_all = {0:{},1:{}} # Initialize an empty map
    root_dir = Path(root_dir)
    files = [file for file in root_dir.iterdir() if file.is_file()]
    for filename_full_path in files:
        filename = Path(filename_full_path).name
        meta_row = metadata[metadata['patientId'] == os.path.splitext(filename)[0]]
        if (len(meta_row)) > 0:
          meta_row_first = meta_row[0]
          label = meta_row_first[5]
          full_path = os.path.join(root_dir, filename)
          if len(file_map_all[label]) != limit:
            file_map_all[label][filename] = full_path
    return file_map_all

start_time = time.time()

metadata = np.genfromtxt(rsna_metadata_path, delimiter=',', names=True,
        dtype='U40,i4,i4,i4,i4,i4')

file_map_all = create_file_map_all(rsna_train_dir, metadata, 100000)

mrnet_file_map = file_map_all[0]
knee_mri_files_map = file_map_all[1]

mrnet_df = pd.DataFrame(mrnet_file_map.items(), columns=['file_name', 'file_path'])
mrnet_label = pd.DataFrame({'label': np.zeros(len(mrnet_df))})
mrnet_df_new = pd.concat([mrnet_df, mrnet_label], axis=1)

knee_mri_df = pd.DataFrame(knee_mri_files_map.items(), columns=['file_name', 'file_path'])
knee_mri_label = pd.DataFrame({'label': np.ones(len(knee_mri_df))})
knee_mri_df_new = pd.concat([knee_mri_df, knee_mri_label], axis=1)

merged_df = pd.concat([mrnet_df_new, knee_mri_df_new], ignore_index=True)

elapsed_time = time.time() - start_time
print(f"Time taken: {elapsed_time:.4f} seconds")

print(merged_df)

Time taken: 138.8557 seconds
                                      file_name  \
0      f79893ef-5f7f-4b63-bcbe-694a2bd8bf42.dcm   
1      f70553b5-ee4f-45f4-a023-0d696aeedffe.dcm   
2      f7b557b1-623e-4466-a60b-ada96f7ba8ff.dcm   
3      f79fa94e-8886-425c-95d4-81abfea92aee.dcm   
4      f69b3d65-3650-4bd3-8809-a0fc0eac5f2a.dcm   
...                                         ...   
26679  09b0df7c-90bc-499d-a7c7-5474199da4e6.dcm   
26680  0930b0aa-25c2-4624-a3fc-9b60ba0b23f4.dcm   
26681  097788d4-cb88-4457-8e71-0ca7a3da2216.dcm   
26682  09326eb7-f4cb-4d8f-83c6-8ba7fb8b5ac7.dcm   
26683  0909a8b0-09e7-4a78-9bfb-417a61c6723c.dcm   

                                               file_path  label  
0      /content/drive/MyDrive/Semester-Docs/Sem5/Caps...    0.0  
1      /content/drive/MyDrive/Semester-Docs/Sem5/Caps...    0.0  
2      /content/drive/MyDrive/Semester-Docs/Sem5/Caps...    0.0  
3      /content/drive/MyDrive/Semester-Docs/Sem5/Caps...    0.0  
4      /content/drive/MyDriv

# Download all tools

In [ ]:
# !git clone https://github.com/subrat-mishra/MedSam-Utils
# import sys
# sys.path.append(root_dir)
# sys.path.append("MedSam-Utils")

In [5]:
!pip3 install tensorboardX
!pip3 install git+https://github.com/bowang-lab/MedSAM.git
!pip install umap-learn
!pip install pydicom
!pip install grad-cam
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.8 MB/s eta 0:00:00
  Cloning https://github.com/bowang-lab/MedSAM.git to /tmp/pip-req-build-bc9aswx7
  Running command git clone --filter=blob:none --quiet https://github.com/bowang-lab/MedSAM.git /tmp/pip-req-build-bc9aswx7
  Resolved https://github.com/bowang-lab/MedSAM.git to commit 91ce9665656c4420b3781f5de87fc4daa7e596a0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 11.1 MB/s eta 0:00:00


In [8]:
# download model and data
img_id = '1Qf0IQF1zTwOJ4Fh4Ix5rtVhR9cnNbI2M'
!gdown $img_id
model_id = "1UAmWL88roYR7wKlnApw5Bcuzf2iQgk6_&confirm=t"
!gdown $model_id

Downloading...
From: https://drive.google.com/uc?id=1Qf0IQF1zTwOJ4Fh4Ix5rtVhR9cnNbI2M
To: /content/img_demo.png
100% 87.9k/87.9k [00:00<00:00, 29.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1UAmWL88roYR7wKlnApw5Bcuzf2iQgk6_
From (redirected): https://drive.google.com/uc?id=1UAmWL88roYR7wKlnApw5Bcuzf2iQgk6_&confirm=t&uuid=7db54240-cd78-413c-9dff-d7d29b695799
To: /content/medsam_vit_b.pth
100% 375M/375M [00:03<00:00, 108MB/s]


# MedSam Embeddings

In [9]:
import csv
import os
import pickle
import numpy as np
import pandas as pd
import pydicom
import time
import torch
import torch.nn as nn
from segment_anything import sam_model_registry
from skimage import io, transform
from PIL import Image
import torch.nn.functional as F

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MedSAM_CKPT_PATH = "medsam_vit_b.pth"
medsam_model = sam_model_registry['vit_b'](checkpoint=MedSAM_CKPT_PATH)
medsam_model = medsam_model.to(device)
medsam_model.eval()

def get_medsam_embeddings(img_3c):
  img_1024 = transform.resize(img_3c, (1024, 1024), order=3, preserve_range=True, anti_aliasing=True).astype(np.uint8)
  img_1024 = (img_1024 - img_1024.min()) / np.clip(
      img_1024.max() - img_1024.min(), a_min=1e-8, a_max=None
  )
  # convert the shape to (3, H, W)
  img_1024_tensor = torch.tensor(img_1024).float().permute(2, 0, 1).unsqueeze(0).to(device)

  with torch.no_grad():
      image_embedding = medsam_model.image_encoder(img_1024_tensor) # (1, 256, 64, 64)
  return image_embedding

def get_knee_mri_embedding(file_path):
  metadata_csv_path = root_dir + "KneeMRI/metadata.csv"
  metadata = np.genfromtxt(metadata_csv_path, delimiter=',', names=True,
        dtype='i4,i4,i4,i4,i4,i4,i4,i4,i4,i4,U20')
  file_name = os.path.basename(file_path)
  exam = metadata[metadata['volumeFilename'] == file_name]
  with open(file_path, 'rb') as file_handler: # Must use 'rb' as the data is binary
      volumetric_data = pickle.load(file_handler)
  z_start = exam['roiZ']
  depth = exam['roiDepth']
  mid_depth = depth if depth == 1 else (int)(depth/2)
  img_np = volumetric_data[z_start + mid_depth, :, :]
  img_np = img_np.reshape(-1, img_np.shape[-1])
  return img_np

def get_image_arr(file_path):
  if file_path.endswith("npy"):
    np_load = np.load(file_path)
    mid = (int)(np_load.shape[0]/2) - 1
    img_np = np_load[mid] # Mid slice
  elif file_path.endswith("pck"):
    # Special case for KneeMRI image
    img_np = get_knee_mri_embedding(file_path)
  elif file_path.endswith("dcm"):
    img_np = pydicom.dcmread(file_path).pixel_array
  else:
    image = Image.open(file_path)
    img_np = np.array(image)

  return img_np

def get_output_tensor(file_path):
  img_np = get_image_arr(file_path)
  if len(img_np.shape) == 2:
      img_3c = np.repeat(img_np[:, :, None], 3, axis=-1)
  else:
      img_3c = img_np
  embeddings = get_medsam_embeddings(img_3c)
  global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
  output_tensor = global_avg_pool(embeddings)
  output_tensor = output_tensor.view(embeddings.size(1))
  return output_tensor.cpu().numpy()

def get_output_tensor_from_np(img_np):
  if len(img_np.shape) == 2:
      img_3c = np.repeat(img_np[:, :, None], 3, axis=-1)
  else:
      img_3c = img_np
  embeddings = get_medsam_embeddings(img_3c)
  global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
  output_tensor = global_avg_pool(embeddings)
  output_tensor = output_tensor.view(embeddings.size(1))
  return output_tensor.cpu().numpy()

def compute_embedding(index, total, file_path, start_time):
  output_tensor_arr = get_output_tensor(file_path)
  if index % 10 == 0:
    print(f"Time taken: {index}/{total} is {time.time() - start_time:.4f} seconds")
  return output_tensor_arr

# Update dataframe with embeddings

In [ ]:
# start_time = time.time()

# merged_df_copy = merged_df.copy()
# batch_size = 100
# file_batch_size = batch_size
# start = 0
# for i in range(start, len(merged_df_copy), batch_size):
#   merged_df_copy.loc[i:i+batch_size, 'embeddings'] = merged_df_copy.loc[i:i+batch_size].apply(lambda row: compute_embedding(row.name, len(merged_df), row['file_path'], start_time), axis=1)

#   if (i + batch_size) % file_batch_size == 0:
#     pkl_file_name = 'rsa_pneumonia.pkl_'+str(i + batch_size)
#     print("Writing file=", pkl_file_name)
#     merged_df_copy.to_pickle(proj_dir+pkl_file_name)

# pkl_file_name = 'rsa_pneumonia.pkl_final'
# print("Writing file=", pkl_file_name)
# merged_df_copy.to_pickle(proj_dir+pkl_file_name)

# merged_df = merged_df_copy

# TSNE graph

In [ ]:
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
# import numpy as np
# import plotly.express as px
# import pandas as pd

# file_names = merged_df['file_name']
# embeddings = merged_df['embeddings']
# labels = merged_df['label'].to_numpy()
# labels = ['Tear' if elem > 0 else 'No Tear' for elem in labels]

# embeddings = np.stack(embeddings)

# tsne = TSNE(n_components=2)

# # Fit the t-SNE model to the embedding feature
# tsne_results = tsne.fit_transform(embeddings)

# df = pd.DataFrame({'x_axis': tsne_results[:, 0], 'y_axis': tsne_results[:, 1],
#                    'labels':labels, 'file_names': file_names},
#                   columns=['x_axis', 'y_axis', 'labels', 'file_names'])
# fig = px.scatter(df, x="x_axis", y="y_axis", color="labels", hover_name="file_names")
# fig.show()

## Debug Images

In [ ]:
# import pickle
# import os
# import re
# import time

# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.patches as patch

# all_files_map = {}
# all_files_map.update(mrnet_file_map)
# all_files_map.update(knee_mri_files_map)

# view_images = ["0057.npy","483367-7.pck"]

# def plot_img(img_3c):
#   image_path = 'output_image.png'
#   plt.imshow(img_3c, cmap='gray')
#   # plt.axis('off')
#   plt.savefig(image_path, bbox_inches='tight', pad_inches=0, dpi=300)
#   plt.show()

# # path to metadata csv file
# for each_view in view_images:
#   file_path = all_files_map.get(each_view)
#   img_3c = get_image_arr(file_path)
#   print(img_3c.shape)
#   plot_img(img_3c)

# Create a simpleNN model with 3 FC layers

---



In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np
from enum import Enum
import torch.nn.functional as F
from torch.autograd import Function

class OutputStrategy(Enum):
  BCE = 1
  CONSTRASTIVE = 2
  BCE_CONSTRASTIVE = 3
  DOMAIN = 4
  BCE_DOMAIN = 5
  CONSTRASTIVE_DOMAIN = 6
  BCE_CONSTRASTIVE_DOMAIN = 7

class ReverseLayerF(Function):
  @staticmethod
  def forward(ctx, x, alpha):
    ctx.alpha = alpha
    return x.view_as(x)

  @staticmethod
  def backward(ctx, grad_output):
    output = grad_output.neg() * ctx.alpha
    return output, None

class SimpleNN(nn.Module):
  def __init__(self, input_size, output_strategy, embeddingEnabled=True):
    super(SimpleNN, self).__init__()
    self.output_strategy = output_strategy
    self.embeddingEnabled = embeddingEnabled

    self.class_classifier = nn.Sequential()
    self.class_classifier.add_module('c_fc1', nn.Linear(input_size, 128))
    self.class_classifier.add_module('c_relu1', nn.ReLU())
    self.class_classifier.add_module('c_fc2', nn.Linear(128, 64))
    self.class_classifier.add_module('c_relu2', nn.ReLU())
    self.class_classifier.add_module('c_fc3', nn.Linear(64, 1))
    self.class_classifier.add_module('c_sigmoid', nn.Sigmoid())

    if OutputStrategy.DOMAIN.name in output_strategy.name:
      self.domain_classifier = nn.Sequential()
      self.domain_classifier.add_module('d_fc1', nn.Linear(input_size, 128))
      self.domain_classifier.add_module('d_relu1', nn.ReLU())
      self.domain_classifier.add_module('d_fc2', nn.Linear(128, 1))
      self.domain_classifier.add_module('d_softmax', nn.Sigmoid())

  def forward(self, x, y=None, alpha=None):
    # if not self.embeddingEnabled:
    #   x = get_output_tensor_from_np(x)
    output1, output2, output3 = None, None, None
    if (OutputStrategy.CONSTRASTIVE.name in output_strategy.name):
      output1 = self.class_classifier(x)
      output2 = self.class_classifier(y)
    elif (OutputStrategy.BCE.name in output_strategy.name):
      output1 = self.class_classifier(x)

    if alpha is not None and OutputStrategy.DOMAIN.name in output_strategy.name:
      reverse_x = ReverseLayerF.apply(x, alpha)
      output3 = self.domain_classifier(reverse_x)
    return output1, output2, output3

class CustomDataset(Dataset):
  def __init__(self, dataframe, is_new_df_req=True):
    self.data = dataframe
    self.is_new_df_req = is_new_df_req

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    filepath = self.data.iloc[idx]['file_path']
    label = torch.tensor(self.data.iloc[idx]['label'], dtype=torch.float32)
    embeddings = torch.from_numpy(self.data.iloc[idx]['embeddings'])
    if not self.is_new_df_req:
      return embeddings, label, filepath, embeddings, label, filepath

    # if any(embeddings.isna()):
    #   embeddings = get_output_tensor(self.data.iloc[idx]['file_path'])

    ## CREATE A NEW DATAFRAME TO AVOID RE-SELECTING ORIGINAL IMAGE
    new_dataframe = self.data
    length = len(new_dataframe)
    target = np.random.randint(0, 2) # Create a random index
    random = np.random
    if target == 1:
      ## GET NEGATIVE COUNTERPART
      label_other = label
      while label_other == label:
        choice = random.choice(length)
        if (choice != idx):  # Ignore the same index
          label_other = new_dataframe.iloc[choice]['label']
    else:
      ## GET POSITIVE COUNTERPART
      label_other = 1.0 - label
      while label_other != label:
        choice = random.choice(length)
        if (choice != idx):   # Ignore the same index
          label_other = new_dataframe.iloc[choice]['label']

    # print(f"Target: {target} length: {length} choice: {choice} otherChoice: {random.choice(length)} label: {label} label_other: {label_other}")

    filepath_ = new_dataframe.iloc[choice]['file_path']
    embeddings_ = torch.from_numpy(new_dataframe.iloc[choice]['embeddings'])
    label_ = torch.tensor(target, dtype=torch.float32)
    return embeddings, label, filepath, embeddings_, label_, filepath_

def evaluate_model(model, loader, threshold, output_strategy):
  model.eval()
  all_predictions, all_labels, file_paths, other_file_paths = [], [], [], []
  with torch.no_grad():
    for inputs1, labels1, file_path1, inputs2, labels2, file_path2 in loader:
      outputs1, outputs2, outputs3 = model(inputs1, inputs2)
      if (OutputStrategy.CONSTRASTIVE.name in output_strategy.name):
        euclidean_distance = F.pairwise_distance(outputs1, outputs2)
        predictions = (euclidean_distance > threshold).float()
        all_labels.extend(labels2.cpu().numpy())
      elif(OutputStrategy.BCE.name in output_strategy.name):
        predictions = (outputs1 > threshold).float()
        all_labels.extend(labels1.cpu().numpy())
      all_predictions.extend(predictions.cpu().numpy())
      file_paths.extend(file_path1)
      other_file_paths.extend(file_path2)
  model.train()
  return all_predictions, all_labels, file_paths, other_file_paths

# Different types of Contrastive loss.


In [5]:
class MarginContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(MarginContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, target):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim=True)
        loss_contrastive = torch.mean((1 - target) * torch.pow(euclidean_distance, 2) +
                                      (target) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

class NormalizedContrastiveLoss(nn.Module):
    def __init__(self):
        super(NormalizedContrastiveLoss, self).__init__()

    def forward(self, output1, output2, target):
        output1_normalized = output1 / output1.norm(dim=1, keepdim=True)
        output2_normalized = output2 / output2.norm(dim=1, keepdim=True)
        cosine_similarity = torch.sum(output1_normalized * output2_normalized, dim=1)
        loss_contrastive = torch.mean((1 - target) * (1 - cosine_similarity) +
                                       target * torch.clamp(cosine_similarity - 0.5, min=0.0))
        return loss_contrastive

# Evaluate the model

In [10]:
# Step 1.0: Declare all parameters
dataset1='rsa_pneumonia.pkl_final'
dataset2='chest_xray_pneumonia_augment.pkl_final'
learning_rate = 0.001
best_val_accuracy = 0.0
best_model_params = None
best_f1 = 0.0
best_epoch_id=0
threshold = 0.4
num_epochs = 100
output_strategy = OutputStrategy.BCE_DOMAIN
batch_size = 32
saved_model_name=output_strategy.name.lower() + "/" + dataset1.split(".")[0] + "_" + dataset2.split(".")[0]
input_size = 256  # Assuming your embeddings are of size 256

In [28]:
# Step 1: Load and preprocess the data
from itertools import cycle
import pickle
import os

with open(proj_dir + dataset1, 'rb') as file:
  df = pickle.load(file)

with open(proj_dir + dataset2, 'rb') as file:
  target_df = pickle.load(file)

# Step 1.1: Encode labels to numerical values (0 and 1)
df['label'] = df['label'].astype('category').cat.codes
target_df['label'] = target_df['label'].astype('category').cat.codes

# Step 1.2: Split the data into train, test, and validation sets
train_df, test_val_df = train_test_split(df, test_size=0.25, random_state=42, stratify=df['label'])
test_df, val_df = train_test_split(test_val_df, test_size=0.5, random_state=42)

# Step 2: Create DataLoader instances for training, validation, and testing
train_dataset = CustomDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = CustomDataset(val_df)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = CustomDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

target_dataset = CustomDataset(target_df)
target_loader = DataLoader(target_dataset, batch_size=batch_size, shuffle=True)

# Step 3: Initialize the model, loss function, and optimizer
model = SimpleNN(input_size, output_strategy)
criterion = nn.BCELoss()
contrastive_criterion = MarginContrastiveLoss()
domain_criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Step 4: Train the model
for epoch in range(num_epochs):
  i = -1
  for train_input, target_input in zip(train_loader if len(train_df) > len(target_loader) else train_loader, cycle(target_loader) if len(train_df) < len(target_loader) else target_loader):
    i += 1
    inputs1, labels1, _, inputs2, labels2, _, inputs3, labels3, _, _, _, _ = train_input[0], train_input[1], train_input[2], train_input[3], train_input[4], train_input[5], target_input[0], target_input[1], target_input[2], target_input[3], target_input[4], target_input[5]
    if len(inputs1) != len(inputs3):
      break
  # for inputs1, labels1, _, inputs2, labels2, _ in train_loader:
    if (OutputStrategy.DOMAIN.name in output_strategy.name):
      p = float(i + epoch * len(train_df)) / num_epochs / len(train_df)
      alpha = 2. / (1. + np.exp(-10 * p)) - 1
      optimizer.zero_grad()
      outputs1, outputs2, outputs3 = model(inputs1, inputs2, alpha)
      outputs_1, outputs_2, outputs_3 = model(inputs3, inputs2, alpha)
    else:
      optimizer.zero_grad()
      outputs1, outputs2, outputs3 = model(inputs1, inputs2)

    if (output_strategy == OutputStrategy.BCE):
      loss = criterion(outputs1, labels1.view(-1, 1))
    elif (output_strategy == OutputStrategy.CONSTRASTIVE):
      loss = contrastive_criterion(outputs1, outputs2, labels2.view(-1, 1))
    elif (output_strategy == OutputStrategy.BCE_CONSTRASTIVE):
      loss = criterion(outputs1, labels1.view(-1, 1))
      loss += contrastive_criterion(outputs1, outputs2, labels2.view(-1, 1))
    elif (output_strategy == OutputStrategy.BCE_DOMAIN):
      loss = criterion(outputs1, labels1.view(-1, 1))
      loss += domain_criterion(outputs3, torch.zeros(batch_size).float().view(-1, 1))
      loss += domain_criterion(outputs_3, torch.ones(batch_size).float().view(-1, 1))
    elif (output_strategy == OutputStrategy.CONSTRASTIVE_DOMAIN):
      loss = contrastive_criterion(outputs1, outputs2, labels2.view(-1, 1))
      loss += domain_criterion(outputs3, torch.zeros(batch_size).float().view(-1, 1))
      loss += domain_criterion(outputs_3, torch.ones(batch_size).float().view(-1, 1))
    elif (output_strategy == OutputStrategy.BCE_CONSTRASTIVE_DOMAIN):
      loss = criterion(outputs1, labels1.view(-1, 1))
      loss += contrastive_criterion(outputs1, outputs2, labels2.view(-1, 1))
      loss += domain_criterion(outputs3, torch.zeros(batch_size).float().view(-1, 1))
      loss += domain_criterion(outputs_3, torch.ones(batch_size).float().view(-1, 1))
    loss.backward()
    optimizer.step()

  val_predictions, val_labels, _, _ = evaluate_model(model, val_loader, threshold, output_strategy)
  # Calculate and print accuracy and F1 score
  val_accuracy = accuracy_score(val_labels, val_predictions)
  val_f1 = f1_score(val_labels, val_predictions)

  # if val_accuracy > best_val_accuracy:
  if val_f1 > best_f1:
    print(f'Epoch [{epoch + 1}/{num_epochs}] - '
        f'Old Accuracy: {best_val_accuracy:.4f}, Old F1 Score: {best_f1:.4f} | '
        f'New Accuracy: {val_accuracy:.4f}, New F1 Score: {val_f1:.4f}')
    best_val_accuracy = val_accuracy
    best_model_params = model.state_dict()
    best_f1 = val_f1
    best_epoch_id = epoch
  else:
    print(f'Epoch [{epoch + 1}/{num_epochs}] - '
        f'Curr Accuracy: {val_accuracy:.4f}, Curr F1 Score: {val_f1:.4f}')

  if epoch - 10 > best_epoch_id:
    break

parent_directory = os.path.dirname(proj_dir + saved_model_name)
if not os.path.exists(parent_directory):
  os.makedirs(parent_directory)

# Save the model with best model params
torch.save(best_model_params, proj_dir + saved_model_name)
test_predictions, test_labels, _, _ = evaluate_model(model, test_loader, threshold, output_strategy)

test_accuracy = accuracy_score(test_labels, test_predictions)
test_f1 = f1_score(test_labels, test_predictions)

print(f"Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}")

Epoch [1/100] - Old Accuracy: 0.0000, Old F1 Score: 0.0000 | New Accuracy: 0.7722, New F1 Score: 0.0524
Epoch [2/100] - Old Accuracy: 0.7722, Old F1 Score: 0.0524 | New Accuracy: 0.8061, New F1 Score: 0.4279
Epoch [3/100] - Old Accuracy: 0.8061, Old F1 Score: 0.4279 | New Accuracy: 0.8058, New F1 Score: 0.5179
Epoch [4/100] - Old Accuracy: 0.8058, Old F1 Score: 0.5179 | New Accuracy: 0.8046, New F1 Score: 0.5553
Epoch [5/100] - Old Accuracy: 0.8046, Old F1 Score: 0.5553 | New Accuracy: 0.7914, New F1 Score: 0.5807
Epoch [6/100] - Curr Accuracy: 0.8118, Curr F1 Score: 0.4861
Epoch [7/100] - Old Accuracy: 0.7914, Old F1 Score: 0.5807 | New Accuracy: 0.7749, New F1 Score: 0.5990
Epoch [8/100] - Curr Accuracy: 0.8085, Curr F1 Score: 0.5528
Epoch [9/100] - Curr Accuracy: 0.8049, Curr F1 Score: 0.5908
Epoch [10/100] - Curr Accuracy: 0.8138, Curr F1 Score: 0.5341
Epoch [11/100] - Curr Accuracy: 0.8106, Curr F1 Score: 0.5473
Epoch [12/100] - Curr Accuracy: 0.8109, Curr F1 Score: 0.5615
Epoch [

# Evaluate the model on a different datasets

In [11]:
import pickle
loaded_model = SimpleNN(input_size, output_strategy, embeddingEnabled=False)
loaded_model.load_state_dict(torch.load(proj_dir + saved_model_name))
loaded_model.eval()

dataset2='chest_xray_pneumonia_augment.pkl_final'
# dataset2='rsa_pneumonia.pkl_final'


# Step 1: Load and preprocess the data
with open(proj_dir+dataset2, 'rb') as file:
  df2 = pickle.load(file)

dataset_2 = CustomDataset(df2)
loader2 = DataLoader(dataset_2, batch_size=batch_size, shuffle=True)

val_predictions, val_labels, _, _ = evaluate_model(loaded_model, loader2, threshold, output_strategy)

# Calculate and print accuracy and F1 score
val_accuracy = accuracy_score(val_labels, val_predictions)
val_f1 = f1_score(val_labels, val_predictions)

print(f"Other Dataset Accuracy: {val_accuracy:.4f}, Other Dataset F1 Score: {val_f1:.4f}")

Other Dataset Accuracy: 0.7502, Other Dataset F1 Score: 0.8457


# GradCam

In [ ]:
import cv2
import pickle
import matplotlib.pyplot as plt
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from torchvision import transforms

def get_gradcam_heatmap(image_path, embeddings, model, target_layer, target_class):
  gradcam = GradCAM(model=model, target_layers=[target_layer])
  img_np = get_image_arr(image_path)
  image = Image.fromarray(img_np)
  transform = transforms.Resize((256, 256))
  image_tensor = transform(torch.tensor(img_np).permute(2,0,1)).to(torch.float32)
  print(target_class)
  heatmap = gradcam(input_tensor=image_tensor, targets=[ClassifierOutputTarget(target_class)])[0:]

  return heatmap

loaded_model = SimpleNN(input_size, output_strategy)
loaded_model.load_state_dict(torch.load(proj_dir + saved_model_name))
loaded_model.eval()

with open(proj_dir+dataset2, 'rb') as file:
  df2 = pickle.load(file)

# first=df2.head(1).to_string(index=False)
first=df2.iloc[0]
image_path=first.file_path
target_class=int(first.label)
embeddings=first.embeddings
target_layer=loaded_model.class_classifier[-1]
heatmap = get_gradcam_heatmap(image_path, embeddings, loaded_model, target_layer, target_class)
visualization = show_cam_on_image(image_path, heatmap, use_rgb=True)
Image.fromarray(visualization)

# Inference

In [6]:
import os
import time
import numpy as np
import pandas as pd
from pathlib import Path
import pickle

inference_dir=root_dir+"chest_xray/test/"
file1 = inference_dir+"NORMAL/IM-0001-0001.jpeg"
file2 = inference_dir+"PNEUMONIA/person1_virus_6.jpeg"

file1_path = Path(file1)
file2_path = Path(file2)

data_dict = {
    'file_name': [file1_path.name, file2_path.name],
    'file_path': [os.path.join('',file1), os.path.join('',file2)],
    'label': [0, 1]
}

inference_df = pd.DataFrame(data_dict)
inference_df_file = root_dir + "inference_df.pkl"
if os.path.exists(inference_df_file):
  with open(inference_df_file, 'rb') as f:
    inference_df = pickle.load(f)
else:
  start_time = time.time()
  inference_df['embeddings'] = inference_df.apply(lambda row: compute_embedding(row.name, len(inference_df), row['file_path'], start_time), axis=1)
  with open(inference_df_file, 'wb') as f:
    pickle.dump(inference_df, f)

In [ ]:
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

dataset1='rsa_pneumonia.pkl_final'
dataset2='chest_xray_pneumonia_augment.pkl_final'
input_size = 256
output_strategy = OutputStrategy.BCE_DOMAIN
threshold = 0.4
saved_model_name=output_strategy.name.lower() + "/" + dataset1.split(".")[0] + "_" + dataset2.split(".")[0]
loaded_model = SimpleNN(input_size, output_strategy, embeddingEnabled=False)
loaded_model.load_state_dict(torch.load(proj_dir + saved_model_name))
loaded_model.eval()

inference_dataset = CustomDataset(inference_df, False)
inference_loader = DataLoader(inference_dataset, batch_size=32, shuffle=True)

val_predictions, val_labels, file_paths, _ = evaluate_model(loaded_model, inference_loader, threshold, output_strategy)
for index, _ in enumerate(val_predictions):
  img = mpimg.imread(file_paths[index])
  plt.imshow(img)
  plt.axis('off')  # Turn off axis labels
  plt.title(f'Expected Label: {val_predictions[index][0]}, Actual Label: {val_labels[index]}')
  plt.show()